In [136]:
import numpy as np
import json
import os
import numpyro

In [137]:
with open('STJuliaIntroBayesian/data/matches_England.json', 'r') as file:
    data = json.load(file)



In [138]:
label = []
matches = []
score = np.zeros((len(data), 2))
for i in range(len(data)):
    label_temp = (data[i]["label"][:-7])
    score_temp = data[i]["label"][-5:]
    home_score = score_temp[0]
    away_score = score_temp[4]
    score[i] = np.array([home_score, away_score])
    for j in range(len(label_temp)):
        if label_temp[j] == "-":
            label1 = label_temp[:j-1]
            label2 = label_temp[j+2:]
            label.append(label1)
            label.append(label2)
            matches.append([label1, label2, home_score, away_score])
label = np.array(label)
# print(label)
label = np.unique(label)
matches = np.array(matches, dtype=object)

In [139]:
def football_matches(home_teams, away_teams, score_home, score_away, teams):
    μatt = np.random.normal(0, 0.1)
    μdef = np.random.normal(0, 0.1)
    σatt = np.random.exponential(1)
    σdef = np.random.exponential(1)
    home = np.random.normal(0, 1)
        
    # Team-specific effects
    
    attack = np.random.normal(μatt, σatt, len(teams))
    defense = np.random.normal(μdef, σdef, len(teams))

    offset = np.mean(attack) + np.mean(defense)
    
    # the number of matches
    n_matches = len(home_teams)

    # scoring rates θ
    θ_home = np.zeros(n_matches)     # or just θ_home = zeros(n_matches)
    θ_away = np.zeros(n_matches)     # or just θ_away = zeros(n_matches)
        
    # Modeling score-rate and scores for each match
    for i in range(n_matches):
        # scoring rate
        home_team_idx = np.argmax(home_teams[i] == teams)
        away_team_idx = np.argmax(away_teams[i] == teams)

        θ_home[i] = home + attack[home_team_idx] + defense[away_team_idx] - offset
        θ_away[i] = attack[away_team_idx] + defense[home_team_idx] - offset

        # scores
        score_home[i] = np.random.poisson(np.exp(θ_home[i]))  # To ensure positive λ for Poisson distribution
        score_away[i] = np.random.poisson(np.exp(θ_away[i]))



In [145]:
football_matches(matches[:,0], matches[:,1], matches[:,2], matches[:,3], label)

import numpyro
import numpyro.infer
from numpyro.infer import MCMC, NUTS
import jax

nuts_kernel = NUTS(matches.copy())
mcmc = MCMC(nuts_kernel, num_warmup=0, num_samples=3000)
mcmc.run(rng_key=jax.random.PRNGKey(0))
posterior_samples = mcmc.get_samples()


ValueError: Expected `fn` to be a Python callable object; instead found type(fn) = <class 'numpy.ndarray'>.